In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import seaborn as sns
import os
import sys

In [52]:
## Order_products
filename_ordprod = "https://raw.githubusercontent.com/tiagottmoraes/Cornershop-s-Data-Science-Test/main/data/order_products.csv?token=GHSAT0AAAAAABOPI7H3MLEWOP67JFG2MFOOYQBLYWA"
df_ordprod = pd.read_csv(filename_ordprod)
df_ordprod.sample(3)


,order_id,product_id,quantity,buy_unit
97153,d76b2db240676d97eb83226b38080a1b,d941f7183f6331a2c7d59cf1333e93c6,2.000,UN
159214,f7507e7fdc2b9720607e93833d3300a3,9b4c496c006368b399b000baa3b0ae18,2.201,KG
113219,e2809ee88ebefd4f195d593f6e4b81c4,a3bfa246d20a8d5f6ea97a4aa0f8781d,1.000,UN


In [63]:
missing_data_order = df_ordprod.isnull()
for column in missing_data_order.columns.values.tolist():
    print(column)
    print(missing_data_order[column].value_counts())
    print("")

order_id
False    198500
Name: order_id, dtype: int64

product_id
False    198500
Name: product_id, dtype: int64

quantity
False    198500
Name: quantity, dtype: int64

buy_unit
False    198500
Name: buy_unit, dtype: int64



In [53]:
## Shopper
filename_shop = "https://raw.githubusercontent.com/tiagottmoraes/Cornershop-s-Data-Science-Test/main/data/shoppers.csv?token=GHSAT0AAAAAABOPI7H2UGR2NWFIPRYHHKGWYQBLZ7Q"
df_shop = pd.read_csv(filename_shop)
df_shop.sample(3)

,shopper_id,seniority,found_rate,picking_speed,accepted_rate,rating
2721,20ce71b6db2ec854931df3f2f3354b8e,6c90661e6d2c7579f5ce337c3391dbb9,0.8491,1.60,1.00,5.00
1504,e333525119a0802de20cdea9a7d8e767,41dc7c9e385c4d2b6c1f7836973951bf,0.8193,2.25,1.00,4.88
1923,604a74c7e290f8e62552f9f2e738f06b,6c90661e6d2c7579f5ce337c3391dbb9,0.8941,1.25,0.88,4.88


In [62]:
missing_data_order = df_shop.isnull()
for column in missing_data_order.columns.values.tolist():
    print(column)
    print(missing_data_order[column].value_counts())
    print("")

shopper_id
False    2864
Name: shopper_id, dtype: int64

seniority
False    2864
Name: seniority, dtype: int64

found_rate
False    2763
True      101
Name: found_rate, dtype: int64

picking_speed
False    2864
Name: picking_speed, dtype: int64

accepted_rate
False    2837
True       27
Name: accepted_rate, dtype: int64

rating
False    2780
True       84
Name: rating, dtype: int64



In [54]:
## Store
filename_store = "https://raw.githubusercontent.com/tiagottmoraes/Cornershop-s-Data-Science-Test/main/data/storebranch.csv?token=GHSAT0AAAAAABOPI7H364D7QLOMEDWNNKECYQBL3AA"
df_store = pd.read_csv(filename_store)
df_store.sample(3)

,store_branch_id,store_id,lat,lng
377,4462bf0ddbe0d0da40e1e828ebebeb11,f718499c1c8cef6730f9fd03c8125cab,-33.400867,-70.594175
313,b0bef4c9a6e50d43880191492d4fc827,a8e864d04c95572d1aece099af852d0a,-33.351164,-70.510430
388,3871bd64012152bfb53fdf04b401193f,c4ca4238a0b923820dcc509a6f75849b,-33.451695,-70.692160


In [61]:
missing_data_order = df_store.isnull()
for column in missing_data_order.columns.values.tolist():
    print(column)
    print(missing_data_order[column].value_counts())
    print("")

store_branch_id
False    476
Name: store_branch_id, dtype: int64

store_id
False    476
Name: store_id, dtype: int64

lat
False    476
Name: lat, dtype: int64

lng
False    476
Name: lng, dtype: int64



In [55]:
## ORDERS - LABEL
filename_ORDER = "https://raw.githubusercontent.com/tiagottmoraes/Cornershop-s-Data-Science-Test/main/data/orders.csv?token=GHSAT0AAAAAABOPI7H2YJSMYIOKU7X7FYNCYQBL3QQ"
df_ORDER = pd.read_csv(filename_ORDER)
df_ORDER.sample(3)

,order_id,lat,lng,promised_time,on_demand,shopper_id,store_branch_id,total_minutes
5795,438c479771f12905489067bf1fff7e23,-33.416225,-70.566432,2019-10-19 16:00:00+00:00,False,720a71d3aef8aaa3a23297c52aa3ebb3,f50a6c02a3fc5a3a5d4d9391f05f3efc,39.648512
5834,c65222c1b166f2592454fb33c08c13e4,-33.412699,-70.576941,2019-10-19 19:00:00+00:00,False,2783c94b04196ea23109611dc56d6bc2,1f0e3dad99908345f7439f8ffabdffc4,158.868917
1414,6a9e435e24d640328206a97080a13795,-33.427659,-70.605131,2019-10-19 01:00:00+00:00,False,fa514581fd5eb2b23df6156b8b1228a9,5751ec3e9a4feab575962e78e006250d,62.059937


In [60]:
missing_data_order = df_ORDER.isnull()
for column in missing_data_order.columns.values.tolist():
    print(column)
    print(missing_data_order[column].value_counts())
    print("")

order_id
False    10000
Name: order_id, dtype: int64

lat
False    10000
Name: lat, dtype: int64

lng
False    10000
Name: lng, dtype: int64

promised_time
False    10000
Name: promised_time, dtype: int64

on_demand
False    10000
Name: on_demand, dtype: int64

shopper_id
False    10000
Name: shopper_id, dtype: int64

store_branch_id
False    10000
Name: store_branch_id, dtype: int64

total_minutes
False    8000
True     2000
Name: total_minutes, dtype: int64

